In [1]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

In [2]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [3]:
memory_service = (
    InMemoryMemoryService()
) 

In [5]:
# Define constants used throughout the notebook
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite"),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
    tools=[
        preload_memory
    ],  # Agent now has access to Memory and can search it whenever it decides to!
)

print("✅ Agent with preload_memory tool created.")

✅ Agent with preload_memory tool created.


In [8]:
session_service = InMemorySessionService()

runner = Runner(
    agent=user_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)
print("✅ Runner created.")

✅ Runner created.


In [9]:
await run_session(
    runner, "My Birthday is on Dec 6th, My favorite color is blue", "birthday-session-01"
)


### Session: birthday-session-01

User > My Birthday is on Dec 6th, My favorite color is blue
Model: > Thanks for sharing! I'll remember that your birthday is on December 6th and your favorite color is blue.


In [11]:
birthday_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="birthday-session-01"
)

await memory_service.add_session_to_memory(birthday_session)

In [12]:
await run_session(
    runner, "When is my birthday?", "birthday-session-02" # This is a different session
)


### Session: birthday-session-02

User > When is my birthday?
Model: > I remember that your birthday is on December 6th.


In [13]:
await run_session(
    runner, "What is my favorite color?", "birthday-session-03" # This is a different session
)


### Session: birthday-session-03

User > What is my favorite color?
Model: > Your favorite color is blue.
